In [1]:
import data_gen as gen
from typing import List
from math import sqrt, log, pow, inf

In [25]:
budget = 50000
website_list = ["111","112","113"]
data = gen.generate_data(budget, website_list, 0.1, 0.3, 0.01)
data


{'111': <data_gen.Website at 0x7f9d789c0c10>,
 '112': <data_gen.Website at 0x7f9d789a8190>,
 '113': <data_gen.Website at 0x7f9d7a7e0b20>}

In [26]:
arms = [data[arm_name] for arm_name in data.keys()]
print(sum(arms[0].realisations))
print(sum(arms[1].realisations))
print(sum(arms[2].realisations))

31820
28646
32008


In [18]:
def calc_beta(arm, a):
    return sqrt(a/arm.num)

In [19]:
def get_max_key(input_dict,keys):
    new_dict = dict()
    for item in keys:
        new_dict[item] = input_dict[item]
    return max(new_dict, key=new_dict.get)


def get_min_key(input_dict, keys):
    new_dict = dict()
    for item in keys:
        new_dict[item] = input_dict[item]
    return min(new_dict, key=new_dict.get)


In [27]:
def select_arm(current_arms, m, a):   
    current_arms = [data[arm_name] for arm_name in data.keys()]
    for arm in current_arms:
        arm.pull()
    ukt_dict = dict()
    lkt_dict = dict()
    bkt_dict = dict()
    beta_dict = dict()
    # Compute Uk(t) and Lk(t) for each arm\n",
    for k in current_arms:
        beta_dict[k] = calc_beta(k, a)
        ukt = k.last_average + beta_dict[k]
        lkt = k.last_average - beta_dict[k]
        ukt_dict[k] = ukt
        lkt_dict[k] = lkt
    # Compute Bk(t) for each arm\n",
    for k in current_arms:
        ukt_dict_subset = ukt_dict.copy()
        ukt_dict_subset.pop(k)
        max_ukt = max(ukt_dict_subset, key=ukt_dict_subset.get)
        bkt = ukt_dict[max_ukt] - lkt_dict[k]
        bkt_dict[k] = bkt

    # Identify the set of m arms J(t)\n",
    bkt_dict_sorted = dict(sorted(bkt_dict.items(), key=lambda item: item[1]))
    Jt = set(list(bkt_dict_sorted.keys())[:m])

    # Identify arm with minimum lower bound in J(t)\n",
    lowest = inf
    contenders = []
    for arm in Jt:
        if lkt_dict[arm] <= lowest:
            contenders.append(arm)
    if len(contenders) > 1:
        lower = get_min_key(beta_dict, contenders)
    else:
        lower = contenders[0]
    # Identify arm with maximum upper bound *not* in J(t)\n",
    not_in_jt = list(set(ukt_dict.keys()) - Jt)
    highest = -inf
    contenders = []
    for arm in not_in_jt:
        if ukt_dict[arm] >= highest:
            contenders.append(arm)
    if len(contenders) > 1:
        upper = get_max_key(beta_dict, contenders)
    else:
        upper = contenders[0]
    # Identify and pull arm I(t)\n",
    keyz = [lower, upper]
    It = get_max_key(beta_dict, keyz)
    It.pull()
    return Jt, bkt_dict


In [28]:
def UGapEb(data, m, budget, a):
    K = len(data)
    ## Initialise
    for arm_name in data.keys():
        data[arm_name].pull()
    ## Main loop
    Jt = ''
    bkt_dict = ''
    for t in range(K+1,budget):
        current_arms = [data[arm_name] for arm_name in data.keys()]
        Jt, bkt_dict = select_arm(current_arms, m, a)
    return get_min_key(bkt_dict, Jt)
    
print(UGapEb(data, 1, 100, 0.1).name)

for website in data.keys():
    print(
        f'website: {data[website].name}, num: {data[website].num}, average: {data[website].average}')
    


113
website: 111, num: 129, average: 0.6356589147286822
website: 112, num: 129, average: 0.4806201550387597
website: 113, num: 129, average: 0.689922480620155


In [36]:
def UGapEc(data, e, m, delta, c):
    K = len(data)
    
    ## Initialise arms
    for arm_name in data.keys():
        data[arm_name].pull()

    Jt = '111'
    Bkt_dict = {Jt:1}
    t = K + 1
    while (Bkt_dict[Jt] >= e):
        current_arms = [data[arm_name] for arm_name in data.keys()]
        a = c * log((4*K*(t**3))/delta)
        Jt, Bkt_dict = select_arm(current_arms, m, a)
        t += 1

    return Jt

In [37]:
print(UGapEc(data, 0.6, 100, 0.9, 0.5).name)

IndexError: list index out of range